In [2]:
!pip install datasets nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which i

In [5]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from datasets import load_dataset

dataset = load_dataset("amazon_polarity", split="train")
df = pd.DataFrame(dataset).sample(n=200, random_state=42)

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

df['sentiment_score'] = df['content'].apply(lambda x: sia.polarity_scores(str(x))['compound'])
df['sentiment_label'] = df['sentiment_score'].apply(lambda x: 'Positive' if x > 0.05 else ('Negative' if x < -0.05 else 'Neutral'))

print(df[['content', 'sentiment_score', 'sentiment_label']].head())


                                                   content  sentiment_score  \
2079998  This product consists of a piece of thin flexi...           0.9041   
1443106  Even on the lowest setting, the toast is too d...          -0.5859   
3463669  I enjoyed this disc. The video is stunning. I ...           0.9545   
2914699  The authors pretend that parents neither die n...          -0.0654   
1603231  Might as well just use a knife, this product h...          -0.1779   

        sentiment_label  
2079998        Positive  
1443106        Negative  
3463669        Positive  
2914699        Negative  
1603231        Negative  


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['content'])
sequences = tokenizer.texts_to_sequences(df['content'])
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')

labels = np.array(df['label'])

split = int(0.8 * len(padded_sequences))
X_train, X_test = padded_sequences[:split], padded_sequences[split:]
y_train, y_test = labels[:split], labels[split:]


model = tf.keras.Sequential([
    tf.keras.layers.Embedding(5000, 64, input_length=100),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=5, batch_size=16, validation_data=(X_test, y_test))

model.save("lstm_sentiment_model_200.h5")


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 140ms/step - accuracy: 0.5237 - loss: 0.6939 - val_accuracy: 0.4250 - val_loss: 0.6964
Epoch 2/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - accuracy: 0.5657 - loss: 0.6904 - val_accuracy: 0.4500 - val_loss: 0.6984
Epoch 3/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - accuracy: 0.5579 - loss: 0.6902 - val_accuracy: 0.4500 - val_loss: 0.6969
Epoch 4/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - accuracy: 0.6144 - loss: 0.6780 - val_accuracy: 0.4500 - val_loss: 0.7012
Epoch 5/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 0.6227 - loss: 0.6417 - val_accuracy: 0.5250 - val_loss: 0.7237


In [7]:

cnn_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(5000, 64, input_length=100),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

cnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

cnn_model.fit(X_train, y_train, epochs=5, batch_size=16, validation_data=(X_test, y_test))
cnn_model.save("cnn_sentiment_model_200.h5")


Epoch 1/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 77ms/step - accuracy: 0.5282 - loss: 0.6929 - val_accuracy: 0.4500 - val_loss: 0.6992
Epoch 2/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.5286 - loss: 0.6635 - val_accuracy: 0.4500 - val_loss: 0.7023
Epoch 3/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.5645 - loss: 0.6292 - val_accuracy: 0.4500 - val_loss: 0.7028
Epoch 4/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9913 - loss: 0.5891 - val_accuracy: 0.4500 - val_loss: 0.7028
Epoch 5/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 1.0000 - loss: 0.5266 - val_accuracy: 0.4500 - val_loss: 0.7055


In [8]:
test_reviews = ["The product is amazing!", "Worst purchase ever!", "It was okay, not great."]
test_sequences = tokenizer.texts_to_sequences(test_reviews)
test_padded = pad_sequences(test_sequences, maxlen=100, padding='post')
predictions = model.predict(test_padded)
pred_labels = ["Positive" if p > 0.5 else "Negative" for p in predictions]


for review, sentiment in zip(test_reviews, pred_labels):
    print(f"Review: {review} | Sentiment: {sentiment}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 395ms/step
Review: The product is amazing! | Sentiment: Negative
Review: Worst purchase ever! | Sentiment: Negative
Review: It was okay, not great. | Sentiment: Negative
